In [1]:
from pyspark.sql import SparkSession

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

In [3]:
spark = SparkSession.builder.master("local[1]").appName("VarunAssessment52").getOrCreate()

:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/varunm15t38hedu/.ivy2/cache
The jars for the packages stored in: /home/varunm15t38hedu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-53c679bf-b826-4cc8-8537-593fb4d048d9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 998ms :: artifacts dl 38ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
order = spark.readStream.format("kafka").option('kafka.bootstrap.servers', 'master:9092').option('subscribe', 'varun_muralidhar').load()

In [6]:
order_data = order.selectExpr("CAST(value AS STRING)") \
    .withColumn("timestamp", split(col("value"), ",")[0].cast("timestamp")) \
    .withColumn("order_id", split(col("value"), ",")[1].cast("string")) \
    .withColumn("user_id", split(col("value"), ",")[2].cast("string")) \
    .withColumn("product_id", split(col("value"), ",")[3].cast("string")) \
    .withColumn("category", split(col("value"), ",")[4].cast("string")) \
    .withColumn("price", split(col("value"), ",")[5].cast("double")) \
    .withColumn("quantity",split(col("value"), ",")[6].cast("integer"))

### 1. Total revenue per 5-minute tumbling window

In [7]:
total_revenue = order_data.groupBy(window("timestamp", "5 minutes")) \
    .agg(sum(order_data.price*order_data.quantity).alias("total_revenue"))

In [ ]:
query = total_revenue.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

25/06/20 08:57:38 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-709a6a9f-c7c5-4c7c-b191-cd05947ae282. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------------+
|window|total_revenue|
+------+-------------+
+------+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-------------+
|              window|total_revenue|
+--------------------+-------------+
|{2025-06-19 00:05...|      4586.85|
|{2025-06-19 00:10...|      4265.69|
+--------------------+-------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-------------+
|              window|total_revenue|
+--------------------+-------------+
|{2025-06-19 00:05...|      4586.85|
|{2025-06-19 00:10...|     10404.59|
|{2025-06-19 00:15...|       2776.0|
+--------------------+-------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----------------+
|              window|    total_revenue|
+--------------------+-----------------+
|{2025-06-19 00:05...|          4586.85|
|{2025-06-19 00:10...|         10404.59|
|{2025-06-19 00:15...|8975.029999999999|
+--------------------+-----------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+-----------------+
|              window|    total_revenue|
+--------------------+-----------------+
|{2025-06-19 00:05...|          4586.85|
|{2025-06-19 00:20...|7207.070000000001|
|{2025-06-19 00:10...|         10404.59|
|{2025-06-19 00:15...|8975.029999999999|
+--------------------+-----------------+



-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------+------------------+
|              window|     total_revenue|
+--------------------+------------------+
|{2025-06-19 00:05...|           4586.85|
|{2025-06-19 00:25...|3903.2200000000003|
|{2025-06-19 00:20...|           9954.19|
|{2025-06-19 00:10...|          10404.59|
|{2025-06-19 00:15...| 8975.029999999999|
+--------------------+------------------+



-------------------------------------------
Batch: 6
-------------------------------------------
+--------------------+-----------------+
|              window|    total_revenue|
+--------------------+-----------------+
|{2025-06-19 00:05...|          4586.85|
|{2025-06-19 00:25...|          11317.5|
|{2025-06-19 00:20...|          9954.19|
|{2025-06-19 00:10...|         10404.59|
|{2025-06-19 00:15...|8975.029999999999|
+--------------------+-----------------+



In [8]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

### 2. Top 3 categories by revenue in a 10-minute sliding window (slide interval = 5 minutes).

In [9]:
revenue_df = order_data.groupBy(window("timestamp", "10 minutes", "5 minutes"), col("category")) \
    .agg(sum(order_data.price*order_data.quantity).alias("revenue"))

In [10]:
order_win = revenue_df.orderBy(col("revenue").desc()).limit(3)

In [ ]:
order_win.writeStream.outputMode("complete").format("console").start().awaitTermination()

25/06/20 09:04:03 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-922ff620-cb6d-4c8d-b7ef-b97532a65521. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+--------+-------+
|window|category|revenue|
+------+--------+-------+
+------+--------+-------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------+-------+
|              window|category|revenue|
+--------------------+--------+-------+
|{2025-06-19 00:45...|   Books|3439.66|
|{2025-06-19 00:50...|   Books|3439.66|
|{2025-06-19 00:45...|    Toys|2277.58|
+--------------------+--------+-------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+--------+-------+
|              window|category|revenue|
+--------------------+--------+-------+
|{2025-06-19 00:50...|   Books|4510.28|
|{2025-06-19 00:50...|    Toys|3667.75|
|{2025-06-19 00:45...|   Books|3439.66|
+--------------------+--------+-------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----------+-----------------+
|              window|   category|          revenue|
+--------------------+-----------+-----------------+
|{2025-06-19 00:50...|      Books|6000.379999999999|
|{2025-06-19 00:55...|Electronics|           5594.3|
|{2025-06-19 00:50...|       Toys|          3667.75|
+--------------------+-----------+-----------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+-----------+-----------------+
|              window|   category|          revenue|
+--------------------+-----------+-----------------+
|{2025-06-19 00:55...|Electronics|           7402.3|
|{2025-06-19 00:50...|      Books|6000.379999999999|
|{2025-06-19 00:55...|   Clothing|          5155.27|
+--------------------+-----------+-----------------+



-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------+-----------+-----------------+
|              window|   category|          revenue|
+--------------------+-----------+-----------------+
|{2025-06-19 00:55...|Electronics|           7402.3|
|{2025-06-19 01:00...|       Toys|          6732.62|
|{2025-06-19 00:50...|      Books|6000.379999999999|
+--------------------+-----------+-----------------+



### 3. Average Order Value (AOV) per user in 1-minute tumbling windows.

In [7]:
average = order_data.groupBy(window("timestamp", "1 minutes"), col("user_id")) \
    .agg(avg(order_data.price*order_data.quantity)).orderBy('user_id').alias("average")

In [ ]:
query = average.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

25/06/20 09:08:18 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b34616b4-3645-4fda-8514-0174d84cfabc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----------------------+
|window|user_id|avg((price * quantity))|
+------+-------+-----------------------+
+------+-------+-----------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-------+-----------------------+
|              window|user_id|avg((price * quantity))|
+--------------------+-------+-----------------------+
|{2025-06-19 01:24...|  U1001|                 2159.2|
|{2025-06-19 01:22...|  U1008|                 990.16|
|{2025-06-19 01:19...|  U1015|                1561.85|
|{2025-06-19 01:23...|  U1020|                1640.05|
|{2025-06-19 01:25...|  U1040|                1982.55|
|{2025-06-19 01:21...|  U1043|                 504.36|
|{2025-06-19 01:20...|  U1047|                 132.81|
|{2025-06-19 01:25...|  U1053|                 121.32|
|{2025-06-19 01:22...|  U1061|                  964.1|
|{2025-06-19 01:19...|  U1061|                 350.44|
|{2025-06-19 01:23...|  U1062|                 1035.8|
|{2025-06-19 01:23...|  U1067|                1065.69|
|{2025-06-19 01:24...|  U1078|                1083.24|
|{2025-06-19 01:19...| 

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-------+-----------------------+
|              window|user_id|avg((price * quantity))|
+--------------------+-------+-----------------------+
|{2025-06-19 01:24...|  U1001|                 2159.2|
|{2025-06-19 01:28...|  U1005|                 598.04|
|{2025-06-19 01:27...|  U1006|     1106.3500000000001|
|{2025-06-19 01:31...|  U1007|                  520.2|
|{2025-06-19 01:30...|  U1008|                 704.05|
|{2025-06-19 01:22...|  U1008|                 990.16|
|{2025-06-19 01:30...|  U1012|                 986.68|
|{2025-06-19 01:26...|  U1012|     1294.8600000000001|
|{2025-06-19 01:19...|  U1015|                1561.85|
|{2025-06-19 01:28...|  U1016|                 451.84|
|{2025-06-19 01:23...|  U1020|                1640.05|
|{2025-06-19 01:25...|  U1023|                 404.69|
|{2025-06-19 01:27...|  U1036|                  920.1|
|{2025-06-19 01:29...| 

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-------+-----------------------+
|              window|user_id|avg((price * quantity))|
+--------------------+-------+-----------------------+
|{2025-06-19 01:24...|  U1001|                 2159.2|
|{2025-06-19 01:37...|  U1002|                 745.11|
|{2025-06-19 01:35...|  U1002|                 1255.5|
|{2025-06-19 01:28...|  U1005|                 598.04|
|{2025-06-19 01:34...|  U1005|                 112.92|
|{2025-06-19 01:27...|  U1006|     1106.3500000000001|
|{2025-06-19 01:31...|  U1007|                  520.2|
|{2025-06-19 01:30...|  U1008|                 704.05|
|{2025-06-19 01:22...|  U1008|                 990.16|
|{2025-06-19 01:33...|  U1009|                1881.44|
|{2025-06-19 01:30...|  U1012|                 986.68|
|{2025-06-19 01:37...|  U1012|                 294.45|
|{2025-06-19 01:26...|  U1012|     1294.8600000000001|
|{2025-06-19 01:19...| 

### 4. Cumulative order count per product.

In [7]:
order_count_pid = order_data.groupBy(order_data.product_id).count()

In [ ]:
query = order_count_pid.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

25/06/20 09:12:57 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-dbff9536-317f-4bd3-a8e6-707048cb812d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|     P5001|    2|
|     P2002|    3|
|     P4001|    2|
|     P4002|    2|
|     P2001|    1|
|     P3001|    1|
+----------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|     P5001|    5|
|     P1002|    1|
|     P2002|    6|
|     P4001|    3|
|     P4002|    2|
|     P3002|    1|
|     P2001|    2|
|     P3001|    1|
+----------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|     P5001|    6|
|     P1001|    1|
|     P1002|    1|
|     P2002|    8|
|     P4001|    3|
|     P4002|    3|
|     P3002|    3|
|     P2001|    4|
|     P3001|    2|
+----------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|     P5001|    8|
|     P1001|    3|
|     P1002|    2|
|     P2002|    8|
|     P5002|    3|
|     P4001|    3|
|     P4002|    3|
|     P3002|    3|
|     P2001|    5|
|     P3001|    2|
+----------+-----+

